In [8]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def scrape_main_content_as_markdown(url, save_path):
    # 下載頁面
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 加入 Metadata 區
    markdown_lines = [
        f"---",
        f"title: Streamlit Documentation Capture",
        f"url: {url}",
        f"date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        f"---\n"
    ]

    # 找到「主內文區域」
    # Streamlit docs 通常主內容會在 <main> 標籤下
    main_content = soup.find('main')
    if not main_content:
        print("⚠️ 找不到 <main> 主內容區，可能頁面結構不同，請確認！")
        return

    # 只抓 <main> 裡的內容（避免 sidebar / footer）
    for element in main_content.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'pre', 'code', 'ul', 'ol', 'li']):
        if element.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            level = int(element.name[1])
            markdown_lines.append(f"{'#' * level} {element.get_text(strip=True)}\n")
        elif element.name == 'p':
            text = element.get_text(strip=True)
            if text:
                markdown_lines.append(text + '\n')
        elif element.name == 'pre':
            code_text = element.get_text()
            markdown_lines.append(f"```python\n{code_text}\n```\n")
        elif element.name == 'code':
            # 單行 code 處理（通常是內文小標記）
            code_inline = element.get_text(strip=True)
            if code_inline:
                markdown_lines.append(f"`{code_inline}`\n")
        elif element.name in ['ul', 'ol']:
            for li in element.find_all('li'):
                li_text = li.get_text(strip=True)
                markdown_lines.append(f"- {li_text}")

    # 合併成一個 markdown 字串
    markdown_content = "\n".join(markdown_lines)

    # 儲存到指定路徑
    with open(save_path, 'w', encoding='utf-8') as f:
        f.write(markdown_content)

    print(f"✅ 內文爬取並儲存完成！已儲存在 {save_path}")

if __name__ == "__main__":
    url = "https://docs.streamlit.io/develop/concepts/multipage-apps/widgets"
    save_path = "/Users/migu/Desktop/資料庫/gen_ai_try/ai_metadata/streamlit/multipage-apps-widgets.md"
    scrape_main_content_as_markdown(url, save_path)


✅ 內文爬取並儲存完成！已儲存在 /Users/migu/Desktop/資料庫/gen_ai_try/ai_metadata/streamlit/multipage-apps-widgets.md
